In [2]:
import pandas as pd

In [4]:
beers_rate = pd.read_csv("../data/RateBeer/beers.csv", sep=",")
breweries_rate = pd.read_csv("../data/RateBeer/breweries.csv", sep=",")
users_rate = pd.read_csv("../data/RateBeer/users.csv", sep=",")